<a href="https://colab.research.google.com/github/Siri2191/Projeto-Senadores/blob/main/Serie_temporal_sobre_senadores_em_exercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Contexto</h1>

---




# Importação das bibliotecas

---

In [18]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go 

# Chamando os dados
---

In [19]:
df =  pd.read_csv('https://raw.githubusercontent.com/Siri2191/Projeto-Senadores/main/df_reembolso_senado_atual.csv')
df.drop(columns=[df.columns[0]], axis=0, inplace=True)
df.head()

,NOME_PARLAMENTAR,PARTIDO,UF,CHEFE_GAB,SEXO,IDADE,MANDATO_INICIO,MANDATO_FINAL,TEMPO_MANDATO,PCT_DE_MANDATO,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,647777,2015-01-14,Sem detalhamento,"52,34",2000857.0
1,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,000647776,2015-01-18,Despesa com pagamento de energia elétrica para...,"168,17",2001140.0
2,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/15,2015-01-30,Despesa com pagamento de aluguel de imóvel par...,5000,2001144.0
3,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,556241490,2015-01-19,Despesa com pagamento de telefonia para uso do...,"416,95",2001141.0
4,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,Aquisição de material de consumo para uso no e...,07.170.542/0001-51,INK JET INFORMATICA E PAPELARIA LTDA EPP,14654,2015-01-09,Despesa com material de consumo para uso do es...,"108,5",2001139.0


In [20]:
len(df)

82010

# Verificando valores nulos

---

In [21]:
df.isnull().sum()

NOME_PARLAMENTAR        0
PARTIDO                 0
UF                      0
CHEFE_GAB               0
SEXO                    0
IDADE                   0
MANDATO_INICIO          0
MANDATO_FINAL           0
TEMPO_MANDATO           0
PCT_DE_MANDATO          0
TIPO_DESPESA            8
CNPJ_CPF                8
FORNECEDOR              8
DOCUMENTO            4260
DATA                   19
DETALHAMENTO            8
VALOR_REEMBOLSADO       8
COD_DOCUMENTO           8
dtype: int64

<h2>Observação:</h2> 

As datas faltantes junto com as informações de fornecedor faltante valem uma análise mais cuidadosa, pois podem indicar algum tipo irregularidade, isto será feita [aqui]()

Outro ponto importante é que há valores de reembolsados faltantes, isto tambem vale uma analise mais cuidadosa.

Por hora esses valores de datas e reembolsado faltantes seram dropados.

As colunas FORNECEDOR, CNPJ_CPF, DOCUMENTO,TIPO_DESPESA, COD_DOCUMENTO, DETALHAMENTO,CHEF_GABINETE serão dropadas, pois não afetará a analise de uma serie temporal. 




In [22]:
df.drop(columns = ['FORNECEDOR','CNPJ_CPF','DOCUMENTO','TIPO_DESPESA','DETALHAMENTO','COD_DOCUMENTO'], axis=0, inplace=True)

In [23]:
df.columns

Index(['NOME_PARLAMENTAR', 'PARTIDO', 'UF', 'CHEFE_GAB', 'SEXO', 'IDADE',
       'MANDATO_INICIO', 'MANDATO_FINAL', 'TEMPO_MANDATO', 'PCT_DE_MANDATO',
       'DATA', 'VALOR_REEMBOLSADO'],
      dtype='object')

In [24]:
# retirando as datas nulas

df.dropna(subset=['DATA'],inplace=True)
df.isnull().sum()

NOME_PARLAMENTAR     0
PARTIDO              0
UF                   0
CHEFE_GAB            0
SEXO                 0
IDADE                0
MANDATO_INICIO       0
MANDATO_FINAL        0
TEMPO_MANDATO        0
PCT_DE_MANDATO       0
DATA                 0
VALOR_REEMBOLSADO    0
dtype: int64

# Verificando tipos

---

In [25]:
df.dtypes

NOME_PARLAMENTAR      object
PARTIDO               object
UF                    object
CHEFE_GAB             object
SEXO                  object
IDADE                  int64
MANDATO_INICIO         int64
MANDATO_FINAL          int64
TEMPO_MANDATO          int64
PCT_DE_MANDATO       float64
DATA                  object
VALOR_REEMBOLSADO     object
dtype: object

In [26]:
df['DATA'] = pd.to_datetime(df['DATA'],format='%Y/%m/%d',errors = 'coerce')  # transformando data em datetime

df.dtypes

NOME_PARLAMENTAR             object
PARTIDO                      object
UF                           object
CHEFE_GAB                    object
SEXO                         object
IDADE                         int64
MANDATO_INICIO                int64
MANDATO_FINAL                 int64
TEMPO_MANDATO                 int64
PCT_DE_MANDATO              float64
DATA                 datetime64[ns]
VALOR_REEMBOLSADO            object
dtype: object

In [27]:
df.head()

,NOME_PARLAMENTAR,PARTIDO,UF,CHEFE_GAB,SEXO,IDADE,MANDATO_INICIO,MANDATO_FINAL,TEMPO_MANDATO,PCT_DE_MANDATO,DATA,VALOR_REEMBOLSADO
0,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,2015-01-14,"52,34"
1,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,2015-01-18,"168,17"
2,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,2015-01-30,5000
3,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,2015-01-19,"416,95"
4,ACIR GURGACZ,PDT,RO,Marcello Augusto Castro Varella,H,60,2015,2023,7,87.5,2015-01-09,"108,5"


In [31]:
df.DATA.min()

Timestamp('1957-12-19 00:00:00')

In [32]:
df.DATA.max()

Timestamp('2041-06-02 00:00:00')

<h2> Observação: </h2> 

* Existem valores de data "estranhos". Estes valores serão dropados e só será feita uma analise entre os valores pedidos entre 2015 e 2022 ( até outubro). 

In [34]:
df.loc[df['DATA'].dt.year<2015]

,NOME_PARLAMENTAR,PARTIDO,UF,CHEFE_GAB,SEXO,IDADE,MANDATO_INICIO,MANDATO_FINAL,TEMPO_MANDATO,PCT_DE_MANDATO,DATA,VALOR_REEMBOLSADO
1866,ALESSANDRO VIEIRA,PSDB,SE,Elaine Da Silva Gontijo,H,47,2019,2027,3,37.5,2002-02-19,2666.43
12473,ESPERIDIÃO AMIN,PP,SC,Robson José De Macedo Gonçalves,H,75,2019,2027,3,37.5,2010-09-12,633.72
13872,FABIANO CONTARATO,PT,ES,Lisandra Melo Barbiero,H,56,2019,2027,3,37.5,2014-07-06,2400.00
16214,FERNANDO BEZERRA COELHO,MDB,PE,-,H,65,2015,2023,7,87.5,1957-12-19,471.29
17287,FERNANDO BEZERRA COELHO,MDB,PE,-,H,65,2015,2023,7,87.5,1957-12-19,471.29
29743,JARBAS VASCONCELOS,MDB,PE,Luno Aurélio De Lima Barbosa,H,80,2019,2027,3,37.5,2014-12-24,628.09
29744,JARBAS VASCONCELOS,MDB,PE,Luno Aurélio De Lima Barbosa,H,80,2019,2027,3,37.5,2014-12-17,142.61
29745,JARBAS VASCONCELOS,MDB,PE,Luno Aurélio De Lima Barbosa,H,80,2019,2027,3,37.5,2014-12-23,88.28
29746,JARBAS VASCONCELOS,MDB,PE,Luno Aurélio De Lima Barbosa,H,80,2019,2027,3,37.5,2014-12-23,149.92
29748,JARBAS VASCONCELOS,MDB,PE,Luno Aurélio De Lima Barbosa,H,80,2019,2027,3,37.5,2014-12-26,121.90


In [36]:
df.loc[df['DATA'].dt.year>2022]

,NOME_PARLAMENTAR,PARTIDO,UF,CHEFE_GAB,SEXO,IDADE,MANDATO_INICIO,MANDATO_FINAL,TEMPO_MANDATO,PCT_DE_MANDATO,DATA,VALOR_REEMBOLSADO
65515,SIMONE TEBET,MDB,MS,Jacqueline Mousinho Macario,M,52,2015,2023,7,87.5,2041-05-07,369.03
65535,SIMONE TEBET,MDB,MS,Jacqueline Mousinho Macario,M,52,2015,2023,7,87.5,2041-06-02,424.35


<h2> Sobre os valores "estranhos": </h2>

* A variavel DATA vem da data presente no documento apresentado par ao pedido de reembolso de um determinado serviço prestado por cnpj OU CPF.  algumas datas dos dados apresentados se apresentam muito antes ou depois do perido temporal que será analisado(2015 - 2022). 

* Das datas anteriores a 2015, destaca-se os valores reembolsado de  [TASSO JEREISSATI](https://www25.senado.leg.br/web/senadores/senador/-/perfil/3396) (senador desde 2002), [PAULO ROCHA](https://www25.senado.leg.br/web/senadores/senador/-/perfil/374)(senador desde 2015) e [JARBAS VASCONCELOS](https://www25.senado.leg.br/web/senadores/senador/-/perfil/4545)(senador desde 2007). 

* Sobre as datas posteriores, se encontra apenas dois valores relacionados a [SIMONE TEBET](https://www25.senado.leg.br/web/senadores/senador/-/perfil/5527)

* Esses valores seram dropados mas vale uma análise cuidadosa posteriomente.  


In [28]:
df['VALOR_REEMBOLSADO'] = df['VALOR_REEMBOLSADO'].str.replace(',','.')  # trocando a virgula por ponto
df['VALOR_REEMBOLSADO'] = df['VALOR_REEMBOLSADO'].astype(float) # convertendo em float

df.dtypes


NOME_PARLAMENTAR             object
PARTIDO                      object
UF                           object
CHEFE_GAB                    object
SEXO                         object
IDADE                         int64
MANDATO_INICIO                int64
MANDATO_FINAL                 int64
TEMPO_MANDATO                 int64
PCT_DE_MANDATO              float64
DATA                 datetime64[ns]
VALOR_REEMBOLSADO           float64
dtype: object

In [29]:
df['VALOR_REEMBOLSADO'].head()

0      52.34
1     168.17
2    5000.00
3     416.95
4     108.50
Name: VALOR_REEMBOLSADO, dtype: float64

# Serie temporal dos valores reembolsados
---

In [30]:
fig = px.line(df, x='DATA', y='VALOR_REEMBOLSADO', title='Serie Temporal dos valores reembolsados')

fig.update_xaxes(rangeslider_visible=True)
fig.show()